In [1]:
from dataclasses import dataclass

Node = tuple[int, int]

@dataclass(frozen=True)
class ResultDictionary:
    """Class representing result of pathfinding algorithm"""
    distance : dict[Node, int]
    path : list[Node]

empty_result = ResultDictionary({}, [])

In [2]:
class Maze:

    def __init__(self, file_name:str, wall_char:str='X', empty_char:str=' '):
        self.maze = []
        self.height = 0
        self.width  =  0
        self.start  = None
        self.end    = None
        self._read_maze(file_name, wall_char, empty_char)

    def _read_maze(self, filename : str, wall_char:str, empty_char:str):
        with open(filename) as f:
            for line in f:
                match line[0]:
                    case 's':
                        self.start = eval('(' + line[6:] + ')') # LISP cursed
                    case 'e':
                        self.end = eval('(' + line[4:] + ')')
                    case _:
                        self.maze.append(list(map(lambda c: c == empty_char, line)))
        self.height = len(self.maze)
        self.width = len(self.maze[0])

    def neighbors(self, n:Node):
        for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
            if self.maze[n[1] + dy][n[0] + dx]:
                yield (n[0] + dx, n[1] + dy)

    def display(self, result:ResultDictionary=empty_result):
        path_set = set(result.path)
        for i in range(self.height):
            for j in range(self.width):
                if not self.maze[i][j]:
                    print('█', end='')
                elif (j, i) == self.start:
                    print('S', end='')
                elif (j, i) == self.end:
                    print('E', end='')
                elif (j, i) in path_set:
                    print('+', end='')
                elif (j, i) in result.distance:
                    print('.', end='')
                else:
                    print(' ', end='')
            print('')
        if result.distance:
            print('Cells opened: ', len(result.distance))
            if result.path:
                print('Path length: ', len(result.path))
            else:
                print('End is unreacheable from start')

In [3]:
m36 = Maze('data/36.txt')
m36.display()

████████████████████████████████████████████████████████
█                                                 █   ██
█  ██ ████   █  █████ █████  █  ████   ████ █ █   █ █ ██
█ █                   █   █   █       █      S█ █ █ █ ██
█   ██ █  ██████ ████ █   ██  ███  █    ██ ███  █ ███ ██
█       █    E          █             █         █     ██
████████████████████████████████████████████████████████


In [4]:
def make_result(maze:Maze, distance:dict[Node, int], pred:dict[Node, Node]):
    path = [maze.end]
    while path[-1] != maze.start:
        path.append(pred[path[-1]])
    return ResultDictionary(distance, path[::-1])

In [5]:
from random import randint

def RandomSearch(maze : Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    tovisit = [maze.start]
    if maze.start == maze.end:
        return ResultDictionary(distance, tovisit)
    
    while tovisit:
        idx = randint(0, len(tovisit) - 1)
        tovisit[idx], tovisit[-1] = tovisit[-1], tovisit[idx]
        current = tovisit.pop()

        for n in maze.neighbors(current):
            if n not in distance:
                tovisit.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])
        

In [6]:
rs = RandomSearch(m36)
m36.display(rs)

████████████████████████████████████████████████████████
█      ..............+++++++++++++++++++++++++....█...██
█  ██ ████...█..█████+█████..█..████...████.█+█...█.█.██
█ █  ...........++++++█...█...█.......█......S█.█.█.█.██
█   ██.█..██████+████.█...██..███..█....██.███..█.███.██
█       █    E+++.......█.............█.........█.....██
████████████████████████████████████████████████████████
Cells opened:  166
Path length:  39


In [7]:
from collections import deque

def BFS(maze : Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    tovisit = deque([maze.start])

    while tovisit:
        current = tovisit.popleft()

        for n in maze.neighbors(current):
            if n not in distance:
                tovisit.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [8]:
bfs = BFS(m36)
m36.display(bfs)

████████████████████████████████████████████████████████
█           ......................................█...██
█  ██ ████   █..█████.█████..█..████...████.█.█...█.█.██
█ █           ........█...█...█.......█++++++S█.█.█.█.██
█   ██ █  ██████.████.█+++██..███..█++++██.███..█.███.██
█       █    E++++++++++█++++++++++++.█.........█.....██
████████████████████████████████████████████████████████
Cells opened:  146
Path length:  37


In [9]:
def DFS(maze:Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    stack = [maze.start]

    while stack:
        current = stack[-1]
        do_pop = True

        for n in maze.neighbors(current):
            if n not in distance:
                do_pop = False
                stack.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
        if do_pop:
            stack.pop()
    return ResultDictionary(distance, [])

In [10]:
dfs = DFS(m36)
m36.display(dfs)

████████████████████████████████████████████████████████
█             .+++++++......++++......++++++.+++. █   ██
█  ██ ████   █.+█████+█████.+█.+████.++████+█+█+. █ █ ██
█ █           .++.  .+█+++█.+.█++++..+█  .++.S█+█ █ █ ██
█   ██ █  ██████+████+█+.+██+.███.+█.+. ██+███.+█ ███ ██
█       █    E+++....+++█++++.....++++█  .++++++█     ██
████████████████████████████████████████████████████████
Cells opened:  120
Path length:  75


In [11]:
from heapq import heappush, heappop
from dataclasses import field
from math import sqrt

@dataclass(order=True)
class PrioritizedNode:
    node: Node=field(compare=False)
    priority: float|int = 0.0
    distance:int=field(compare=False, default=0)
    
    def __iter__(self):
        return iter((self.node, self.distance))

def L1_dist(maze : Maze):
    def l1(n : Node):
        return abs(n[0] - maze.end[0]) + abs(n[1] - maze.end[1])
    return l1

def L2_dist(maze : Maze):
    def l2(n : Node):
        return sqrt((n[0] - maze.end[0])**2 + (n[1] - maze.end[1])**2)
    return l2

def Chebyshev_dist(maze : Maze):
    def chebyshev(n : Node):
        return max((n[0] - maze.end[0]), abs(n[1] - maze.end[1]))
    return chebyshev

In [12]:
def GreedySearch(maze:Maze, heuristic) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    heap = [PrioritizedNode(maze.start)]

    while heap:
        current = heappop(heap).node
        for n in maze.neighbors(current):
            if n not in distance:
                pred[n] = current
                distance[n] = distance[current] + 1
                heappush(heap, PrioritizedNode(n, priority=heuristic(n)))
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [13]:
gs = GreedySearch(m36, Chebyshev_dist(m36))
m36.display(gs)

████████████████████████████████████████████████████████
█             .++++++++++++++++.                  █   ██
█  ██ ████...█.+█████.█████..█++████.  ████.█.█   █ █ ██
█ █     .+++++++.     █   █   █++++++.█++++++S█ █ █ █ ██
█   ██ █.+██████ ████ █   ██  ███..█++++██.███  █ ███ ██
█       █++++E          █           ..█.        █     ██
████████████████████████████████████████████████████████
Cells opened:  71
Path length:  49


In [14]:
def Astar(maze:Maze, heuristic) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    heap = [PrioritizedNode(maze.start)]

    while heap:
        current, current_distance = heappop(heap)
        if current_distance > distance[current]:
            continue
        for n in maze.neighbors(current):
            if n not in distance or distance[n] > distance[current] + 1:
                pred[n] = current
                distance[n] = distance[current] + 1
                heappush(heap, PrioritizedNode(n, distance=distance[n], priority=heuristic(n)+distance[n]))
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [15]:
astar = Astar(m36, L1_dist(m36))
m36.display(astar)

████████████████████████████████████████████████████████
█                                          . .    █   ██
█  ██ ████   █  █████ █████  █ .████.. ████.█.█   █ █ ██
█ █                   █...█ ..█.......█++++++S█ █ █ █ ██
█   ██ █  ██████.████.█+++██..███..█++++██.███  █ ███ ██
█       █    E++++++++++█++++++++++++.█......   █     ██
████████████████████████████████████████████████████████
Cells opened:  70
Path length:  37
